# Task1. 预测是酶还是非酶

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-08-26  

## 任务简介
该任务通过给定蛋白序列，预测该该蛋白是酶还是非酶。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的蛋白序列数据预测是酶还是非酶。


## 数据统计
- 数据源Sprot，酶 219,227 条， 非酶226,539条。
- 将数据集中的所有数据按照时间排序，～90%作为训练集，～10%作为测试集，找到对应时间节点为2009年12月14日。
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 

###  三级以下EC号的算作是酶 







## 1. 导入必要的包、定义公共函数

In [1]:
import numpy as np
import pandas as pd
import random
import time
import datetime
import sys
import os
from tqdm import tqdm
from functools import reduce

sys.path.append("../../tools/")
import commontools
import funclib
import ucTools

sys.path.append("../../")
import benchmark_common as bcommon
import benchmark_train as btrain
import benchmark_test as btest
import benchmark_config as cfg
import benchmark_evaluation as eva

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize() # 初始化该这个b...并行库

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. 加载数据

In [2]:
train = pd.read_feather(cfg.DATADIR+'train.feather')
test = pd.read_feather(cfg.DATADIR+'test.feather')

print('train size: {0}\ntest size: {1}'.format(len(train), len(test)))

trainset = train[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)

train size: 469129
test size: 8033


In [3]:
train.head(2)

,id,isemzyme,functionCounts,ec_number,ec_specific_level,seq,seqlength,f1,f2,f3,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,A0A023GPI8,False,0,-,0,ADTIVAVELDTYPNTDIGDPSYPHIGIDIKSVRSKKTAKWNMQNGK...,236,0.000115,0.105367,0.037413,...,0.052682,-0.020702,0.081852,0.008791,-0.036727,-0.027552,-0.254128,0.020778,0.140904,0.140635
1,A0A023GPJ0,False,0,-,0,MFGIFSKGEPVSMEGELVQPSSIVINDYEEELHLPLSYWDIKDYKN...,143,0.001688,-0.017209,0.029518,...,0.040570,-0.031992,-0.174783,0.000952,-0.136715,0.042773,0.132279,0.015899,0.152930,-0.003412


## 3. 同源比对

In [4]:
# 读取比对结果
res_data=funclib.getblast_fasta(trainfasta=cfg.TRAIN_FASTA,testfasta=cfg.TEST_FASTA)
print(' query samples:{0}\n results samples: {1}'.format(len(test), len(res_data)))

diamond makedb --in /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/train.fasta -d /tmp/train.dmnd


diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database input file: /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/train.fasta
Opening the database file...  [0.041s]
Loading sequences...  [1.241s]
Masking sequences...  [0.444s]
Writing sequences...  [0.148s]
Hashing sequences...  [0.042s]
Loading sequences...  [0s]
Writing trailer...  [0.002s]
Closing the input file...  [0.002s]
Closing the database file...  [0.035s]
Database hash = 472d261fad8ef4b7e6ad2f98871426e9
Processed 469129 sequences, 176771473 letters.
Total time = 1.957s
diamond v2.0.8.146 (C) Max Planck Society for the Advancement of Science
Documentation, support and updates available at http://www.diamondsearch.org

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary direct

diamond blastp -d /tmp/train.dmnd  -q  /home/shizhenkun/codebase/BioUniprot/data/benchmark/data/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


 [0.307s]
Opening the output file...  [0s]
Loading query sequences...  [0.028s]
Masking queries...  [0.038s]
Building query seed set...  [0.039s]
The host system is detected to have 270 GB of RAM. It is recommended to increase the block size for better performance using these parameters : -b8 -c1
Algorithm: Double-indexed
Building query histograms...  [0.017s]
Allocating buffers...  [0s]
Loading reference sequences...  [0.287s]
Masking reference...  [0.436s]
Initializing temporary storage...  [0s]
Building reference histograms...  [0.261s]
Allocating buffers...  [0s]
Processing query block 1, reference block 1/1, shape 1/2.
Building reference seed array...  [0.187s]
Building query seed array...  [0.014s]
Computing hash join...  [0.084s]
Building seed filter...  [0.002s]
Searching alignments...  [0.056s]
Processing query block 1, reference block 1/1, shape 2/2.
Building reference seed array...  [0.139s]
Building query seed array...  [0.01s]
Computing hash join...  [0.089s]
Building seed

 query samples:8033
 results samples: 6034


 [1.041s]
Deallocating reference...  [0.015s]
Loading reference sequences...  [0s]
Deallocating buffers...  [0.003s]
Deallocating queries...  [0s]
Loading query sequences...  [0s]
Closing the input file...  [0s]
Closing the output file...  [0s]
Closing the database file...  [0.004s]
Deallocating taxonomy...  [0s]
Total time = 3.375s
Reported 6034 pairwise alignments, 6034 HSPs.
6034 queries aligned.
The host system is detected to have 270 GB of RAM. It is recommended to increase the block size for better performance using these parameters : -b8 -c1


In [5]:
# 给比对结果添加标签
isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
res_data['isEmzyme_pred'] = res_data['sseqid'].apply(lambda x: isEmzyme_dict.get(x))

isEmzyme_dict_test = {v: k for k,v in zip(test.isemzyme, test.id )} 
res_data['isEmzyme_groudtruth'] = res_data['id'].apply(lambda x: isEmzyme_dict_test.get(x))

# 获取比对结果
res_data_sub = res_data[['id','isEmzyme_pred']]

#选取测试集的ID与是否是酶2列
test_data_sub=test[['id', 'isemzyme']]
test_data_sub.reset_index(drop=True, inplace=True)

#拼合比对结果到测试集
res = pd.merge(test_data_sub,res_data_sub, on='id', how='left')

print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', '\t confusion Matrix')
eva.caculateMetrix(groundtruth=res.isemzyme, predict=res.isEmzyme_pred, baselineName='Blast', type='include_unfind')


baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 	 confusion Matrix
Blast 		0.703598 	0.969148 		0.904980 	0.807488 	0.880963 	 tp: 2890 fp: 92 fn: 290 tn: 2762 up: 399 un: 1600


## 4. 机器学习方法预测
### 4.1 one-hot + 机器学习

In [6]:
MAX_SEQ_LENGTH = 1500 #定义序列最长的长度
trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))


f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

In [ ]:
# 计算指标
X_train = np.array(f_train.iloc[:,2:])
X_test = np.array(f_test.iloc[:,2:])
Y_train = np.array(f_train.isemzyme.astype('int'))
Y_test = np.array(f_test.isemzyme.astype('int'))
funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 		 confusion Matrix
knn 		0.633014 	0.668626 		0.622201 	0.349539 	0.459083 	 tp: 1251 fp: 620 fn: 2328 tn: 3834
lr 		0.649819 	0.590758 		0.715185 	0.696563 	0.639313 	 tp: 2493 fp: 1727 fn: 1086 tn: 2727


/home/shizhenkun/anaconda3/envs/py38/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


## 5. Unirep + 机器学习

In [ ]:
X_train = np.array(train.iloc[:,7:])
X_test = np.array(test.iloc[:,7:])

Y_train = np.array(train.isemzyme.astype('int')).flatten()
Y_test = np.array(test.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

## 6. ESM + 机器学习
### 6.1 REP33

In [ ]:
train_esm_33 = pd.read_feather(cfg.DATADIR + 'train_rep33.feather')
test_esm_33 = pd.read_feather(cfg.DATADIR + 'test_rep33.feather')

train_esm = trainset.merge(train_esm_33, on='id', how='left')
test_esm = testset.merge(test_esm_33, on='id', how='left')

In [ ]:
X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

### 6.2 REP32

In [ ]:
train_esm_32 = pd.read_feather(cfg.DATADIR + 'train_rep32.feather')
test_esm_32 = pd.read_feather(cfg.DATADIR + 'test_rep32.feather')

train_esm = trainset.merge(train_esm_32, on='id', how='left')
test_esm = testset.merge(test_esm_32, on='id', how='left')

X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

### 6.3 REP0

In [ ]:
train_esm_0 = pd.read_feather(cfg.DATADIR + 'train_rep0.feather')
test_esm_0 = pd.read_feather(cfg.DATADIR + 'test_rep0.feather')

train_esm = trainset.merge(train_esm_0, on='id', how='left')
test_esm = testset.merge(test_esm_0, on='id', how='left')

X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

## 7. Ours

In [ ]:
# get blast results
blastres=pd.DataFrame()
blastres['id']=res.id
blastres['isemzyme_groundtruth']=res.isemzyme
blastres['isEmzyme_pred_blast']=res.isEmzyme_pred

In [ ]:
#res32
train_esm_32 = pd.read_feather(cfg.DATADIR + 'train_rep32.feather')
test_esm_32 = pd.read_feather(cfg.DATADIR + 'test_rep32.feather')

train_esm = trainset.merge(train_esm_32, on='id', how='left')
test_esm = testset.merge(test_esm_32, on='id', how='left')

X_train = np.array(train_esm.iloc[:,4:])
X_test = np.array(test_esm.iloc[:,4:])

Y_train = np.array(train_esm.isemzyme.astype('int')).flatten()
Y_test = np.array(test_esm.isemzyme.astype('int')).flatten()

In [ ]:
groundtruth, predict, predictprob = funclib.xgmain(X_train, Y_train, X_test, Y_test, type='binary')
blastres['isEmzyme_pred_xg'] = predict
blastres.isEmzyme_pred_xg =blastres.isEmzyme_pred_xg.astype('bool')
blastres['isEmzyme_pred_slice']=blastres.apply(lambda x: x.isEmzyme_pred_xg if str(x.isEmzyme_pred_blast)=='nan' else x.isEmzyme_pred_blast, axis=1)
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', '\t confusion Matrix')
eva.caculateMetrix( groundtruth=blastres.isemzyme_groundtruth,  predict=blastres.isEmzyme_pred_slice, baselineName='ours', type='binary')

In [62]:
# 保存文件
blastres.to_csv(cfg.FILE_SLICE_ISENZYME_RESULTS, sep='\t', index=None)